In [ ]:
import tensorflow as tf
import numpy as np
import random
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import pandas as pd
from transformers import TFBertModel
import os
import tensorflow_addons as tfa
import keras_tuner as kt

In [1]:
%cd "C:\Users\siddh\Downloads\train_watson.csv"
train_df = pd.read_csv("train.csv")a
train_df

C:\Users\siddh\Downloads\train_watson.csv


,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1
...,...,...,...,...,...,...
12115,2b78e2a914,The results of even the most well designed epi...,All studies have the same amount of uncertaint...,en,English,2
12116,7e9943d152,But there are two kinds of the pleasure of do...,But there are two kinds of the pleasure of doi...,en,English,0
12117,5085923e6c,The important thing is to realize that it's wa...,"It cannot be moved, now or ever.",en,English,2
12118,fc8e2fd1fe,At the west end is a detailed model of the who...,The model temple complex is at the east end.,en,English,2


In [2]:
%cd "C:\Users\siddh\Downloads\test.csv~"
test_df = pd.read_csv("test.csv")
test_df

C:\Users\siddh\Downloads\test.csv~


,id,premise,hypothesis,lang_abv,language
0,c6d58c3f69,بکس، کیسی، راہیل، یسعیاہ، کیلی، کیلی، اور کولم...,"کیسی کے لئے کوئی یادگار نہیں ہوگا, کولمین ہائی...",ur,Urdu
1,cefcc82292,هذا هو ما تم نصحنا به.,عندما يتم إخبارهم بما يجب عليهم فعله ، فشلت ال...,ar,Arabic
2,e98005252c,et cela est en grande partie dû au fait que le...,Les mères se droguent.,fr,French
3,58518c10ba,与城市及其他公民及社区组织代表就IMA的艺术发展进行对话&amp,IMA与其他组织合作，因为它们都依靠共享资金。,zh,Chinese
4,c32b0d16df,Она все еще была там.,"Мы думали, что она ушла, однако, она осталась.",ru,Russian
...,...,...,...,...,...
5190,5f90dd59b0,نیند نے وعدہ کیا کہ موٹل نے سوال میں تحقیق کی.,نیمیتھ کو موٹل کی تفتیش کے لئے معاوضہ دیا جارہ...,ur,Urdu
5191,f357a04e86,The rock has a soft texture and can be bough...,The rock is harder than most types of rock.,en,English
5192,1f0ea92118,她目前的存在，并考虑到他与沃佛斯顿争执的本质，那是尴尬的。,她在与Wolverstone的打斗结束后才在场的事实被看作是很尴尬的。,zh,Chinese
5193,0407b48afb,isn't it i can remember i've only been here ei...,I could see downtown Dallas from where I lived...,en,English


In [3]:
train_df.label.value_counts()

0    4176
2    4064
1    3880
Name: label, dtype: int64

In [1]:
classes = ["entailment","neutral","contradiction"]

In [4]:
random_index = random.randint(0,len(train_df)-5)    # create random index not more than 
for row in train_df[["premise","hypothesis","label"]][random_index:random_index+5].itertuples():
    _, premise, hypothesis, label = row    # _ is to get rid of index
    
    if label == 0: print(f"label: {label}", "{entailment}")
    elif label == 1: print(f"label: {label}", "{neutral}")
    elif label == 2: print(f"label: {label}", "{contradictory}")
    print(f"-> Premise: {premise}\n-> Hypothesis: {hypothesis}\n")
    print("-"*30,"\n")

label: 1 {neutral}
-> Premise: Ouais, eh bien, le mec est là.
-> Hypothesis: Le type est arrivé il y a deux minutes.

------------------------------ 

label: 0 {entailment}
-> Premise: con thứ hai mà cô ấy có là một trong những chú chó từ đống rác và ừ là con đực từng có vấn đề về răng
-> Hypothesis: Người đàn ông có vấn đề về răng và con chó con thứ hai  từ bãi rác.

------------------------------ 

label: 1 {neutral}
-> Premise: And these are tough times for reviewers in general.
-> Hypothesis: Specialist reviewers or reviewers with seniority are having an easier time of it.

------------------------------ 

label: 2 {contradictory}
-> Premise: 'It's that kind of world.'
-> Hypothesis: This was unusual for the world.

------------------------------ 

label: 0 {entailment}
-> Premise: 6See also Internal Control Management and Evaluation Tool (GAO-01-1008G, August 2001).
-> Hypothesis: The tool for control management.

------------------------------ 



In [5]:
train_data, test_data, train_labels, test_labels = train_test_split(train_df[["premise","hypothesis"]].to_numpy(),
                                                                    train_df["label"].to_numpy(),
                                                                    test_size = 0.1,
                                                                    random_state = 42)

C:\Users\siddh\anaconda3\envs\tf_gpu\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
train_data[0], train_labels[0]

(array(["No, I don't know. ", 'Yes, I know.'], dtype=object), 2)

In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')

def encode_sentence(s):
    tokens = list(tokenizer.tokenize(s))
    tokens.append('[SEP]')
    return tokenizer.convert_tokens_to_ids(tokens)

In [14]:
# Testing the tokenize function

print('Original sentence: ' + train_df.premise[0] + '\n')
print('Tokenized sentence: ' + str(encode_sentence(train_df.premise[0])))

Original sentence: and these comments were considered in formulating the interim rules.

Tokenized sentence: [10110, 11269, 49602, 10342, 14553, 10104, 14751, 12305, 10103, 42733, 18634, 119, 102]


In [15]:
# Encode data for the bert model with a max length of 100

def bert_encode(hypotheses, premises, tokenizer, max_length=100):

    x = [h + ' [SEP] ' + p for h, p in zip(np.array(hypotheses), np.array(premises))]
    x = tokenizer(x, padding=True, truncation=True, max_length=max_length)

    inputs = {
          'input_word_ids':tf.ragged.constant(x['input_ids']).to_tensor(),
          'input_mask': tf.ragged.constant(x['attention_mask']).to_tensor(),
          'input_type_ids': tf.ragged.constant(x['token_type_ids']).to_tensor()}

    return inputs

In [17]:
train_input = bert_encode(train_df.premise.values, train_df.hypothesis.values, tokenizer)

train_input

{'input_word_ids': <tf.Tensor: shape=(12120, 100), dtype=int32, numpy=
 array([[  101, 10110, 11269, ...,     0,     0,     0],
        [  101, 11269, 10320, ...,     0,     0,     0],
        [  101, 10143, 29761, ...,     0,     0,     0],
        ...,
        [  101, 10103, 12652, ...,     0,     0,     0],
        [  101, 10160, 10103, ...,     0,     0,     0],
        [  101, 10139, 14885, ...,     0,     0,     0]])>,
 'input_mask': <tf.Tensor: shape=(12120, 100), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]])>,
 'input_type_ids': <tf.Tensor: shape=(12120, 100), dtype=int32, numpy=
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])>}

In [18]:
test_input = bert_encode(test_df.premise.values, test_df.hypothesis.values, tokenizer)

test_input

{'input_word_ids': <tf.Tensor: shape=(5195, 100), dtype=int32, numpy=
 array([[  101,   452, 28821, ...,     0,     0,     0],
        [  101, 13812, 12153, ...,     0,     0,     0],
        [  101, 10137, 17779, ...,     0,     0,     0],
        ...,
        [  101,  3018,  6017, ...,     0,     0,     0],
        [  101, 65148,   112, ...,     0,     0,     0],
        [  101, 10104, 14001, ...,     0,     0,     0]])>,
 'input_mask': <tf.Tensor: shape=(5195, 100), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]])>,
 'input_type_ids': <tf.Tensor: shape=(5195, 100), dtype=int32, numpy=
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])>}

In [19]:
# Enable TPU for faster training

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

Number of replicas: 1


In [20]:
#from official.nlp import optimization
os.environ["WANDB_API_KEY"] = "0" # to silence warning...sometimes :D

max_len = 100
bert_encoder = TFBertModel.from_pretrained('bert-base-multilingual-cased')

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [21]:
def build_model(hp,base_model=bert_encoder):
    '''
    Keras tunes model using bert_encoder(base_multilingual_cased) and finding out the optimal hyperparameters
    '''
    base_model.trainable = False
    
# INPUTS
    input_word_ids = tf.keras.Input(shape=(max_len,),dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,),dtype=tf.int32, name="input_mask")
    input_type_ids = tf.keras.Input(shape=(max_len,),dtype=tf.int32, name="input_type_ids")
    
    x = base_model([input_word_ids, input_mask, input_type_ids])[0]
    x = tf.keras.layers.LSTM(units=hp.Int('lstm_', min_value=50, max_value=100), return_sequences=True)(x)
    for i in range(hp.Int('num_dense_layers', 1, 5)):
        x = tf.keras.layers.Dropout(hp.Choice('dropout_', values=[0.0, 0.1, 0.2]))(x)
        x = tf.keras.layers.Dense(units=hp.Int('dense_', min_value=50, max_value=100), activation='relu')(x)

# OUTPUT
    output = tf.keras.layers.Dense(3, activation='softmax')(x[:,0,:])

# BUILD THE FRAMEWORK
    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=output)

# OPTIMIZER
    step = tf.Variable(0, trainable=True)
    schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
    [5000, 7500,10000,12500], [1e-0, 5e-1, 1e-1, 5e-2,1e-2])
    lr = 1e-3 * schedule(step)
    wd = lambda: 1e-4 * schedule(step)

    optimizer = tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd)

# COMPILE
    model.compile(optimizer=optimizer, 
                  loss= tf.keras.losses.SparseCategoricalCrossentropy(), 
                  metrics=["accuracy",f1])
    
    return model

In [26]:
# train BERT model

tuner = kt.tuners.BayesianOptimization(build_model,
                                        seed=42,
                                        objective='val_loss',
                                        max_trials=5,
                                        directory='.',
                                        project_name = "My_dear_Watson1")

tuner.search(train_input,
            train_df.label.values,
            epochs = 12,
            verbose = 1,
            batch_size = 64,
            validation_split = 0.25)

Trial 5 Complete [00h 09m 46s]
val_loss: 1.0387625694274902

Best val_loss So Far: 1.0330836772918701
Total elapsed time: 00h 48m 58s
INFO:tensorflow:Oracle triggered exit


In [3]:
best_model = tuner.get_best_models(1)[0]
best_model.summary()

NameError: name 'tuner' is not defined

In [ ]:
for layer_number, layer in enumerate(best_model.layers):
    print(f"{layer_number}  {layer.name} : {layer.trainable}")

In [29]:
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]
best_hyperparameters

In [30]:
best_model.fit(train_input,
            train_df.label.values,
            epochs = 15,
            verbose = 1,
            batch_size = 64,
            validation_split = 0.25)

Epoch 1/15
143/143 [==============================] - 57s 337ms/step - loss: 1.0148 - accuracy: 0.4776 - f1: 1.6319 - val_loss: 1.0365 - val_accuracy: 0.4594 - val_f1: 1.7414
Epoch 2/15
143/143 [==============================] - 46s 324ms/step - loss: 1.0124 - accuracy: 0.4868 - f1: 1.5885 - val_loss: 1.0351 - val_accuracy: 0.4680 - val_f1: 1.9545
Epoch 3/15
143/143 [==============================] - 47s 327ms/step - loss: 1.0119 - accuracy: 0.4859 - f1: 1.6419 - val_loss: 1.0407 - val_accuracy: 0.4779 - val_f1: 1.5509
Epoch 4/15
143/143 [==============================] - 47s 332ms/step - loss: 1.0088 - accuracy: 0.4883 - f1: 1.5803 - val_loss: 1.0410 - val_accuracy: 0.4686 - val_f1: 1.5110
Epoch 5/15
143/143 [==============================] - 47s 331ms/step - loss: 1.0026 - accuracy: 0.4938 - f1: 1.5731 - val_loss: 1.0362 - val_accuracy: 0.4607 - val_f1: 1.7209
Epoch 6/15
143/143 [==============================] - 47s 332ms/step - loss: 0.9986 - accuracy: 0.4974 - f1: 1.5459 - val_los

In [2]:
def pred_class(sample_premise,sample_hypothesis,tokenizer):
    '''
    Predicts classes for the sample input
    '''
    
    global classes
    sample_input = bert_encode(sample_premise,sample_hypothesis,tokenizer)
    pred = model.predict(sample_input)
    return classes[tf.argmax(pred,axis=1)]